In [1]:
# For usage on google colab:
!git clone https://github.com/ahans30/Binoculars.git
%cd Binoculars
!pip install -e .

Cloning into 'Binoculars'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 208 (delta 62), reused 55 (delta 55), pack-reused 138 (from 1)
Receiving objects: 100% (208/208), 54.08 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Updating files: 100% (27/27), done.
/content/Binoculars
Obtaining file:///content/Binoculars
  Preparing metadata (setup.py) ... done
     \ 17.4 MB 9.6 MB/s 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from binoculars import Binoculars

In [3]:
import numpy as np
import torch
import transformers
import pickle as pkl
import pandas as pd
from google.colab import files

In [4]:
bino = Binoculars()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


In [9]:
df = pd.read_csv("/content/raid_test_100.csv")

In [10]:
df.head()

,Unnamed: 0,generation,attack,domain,repetition_penalty,model
0,0,Segmentations-Leak: Membership Inference Attac...,perplexity_misspelling,abstracts,no,mistral-chat
1,1,"Farshid Jamshidian is a finance researcher, ac...",insert_paragraphs,wiki,NaN,human
2,2,Transient spine enlargement (3-5 min timesca...,alternative_spelling,abstracts,NaN,human
3,3,The viola da Terceira (also viola Terceiren...,whitespace,wiki,NaN,human
4,4,Yes this review is 2 years after the film's re...,insert_paragraphs,reviews,NaN,human


In [11]:
texts = df[['generation']]
print(texts)

                                           generation
0   Segmentations-Leak: Membership Inference Attac...
1   Farshid Jamshidian is a finance researcher, ac...
2     Transient spine enlargement (3-5 min timesca...
3   The   viola da Terceira  (also viola Terceiren...
4   Yes this review is 2 years after the film's re...
..                                                ...
95    We consider the orbit type filtration on a m...
96  George Michael is to perform live at London's ...
97  Nicole Kelly is an Australian contemporary Chr...
98  In this paper, we propose a novel approach to ...
99  If   life is still bad for me even  when I com...

[100 rows x 1 columns]


In [24]:
batch = texts['generation'].tolist()[0:10]
encodings = bino._tokenize(batch)
observer_logits, performer_logits = bino._get_logits(encodings)
pad_token = bino.tokenizer.pad_token_id

In [25]:
ce_loss_fn = torch.nn.CrossEntropyLoss(reduction="none")
softmax_fn = torch.nn.Softmax(dim=-1)
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [26]:
def perplexity(encoding: transformers.BatchEncoding,
               logits: torch.Tensor,
               median: bool = False,
               temperature: float = 1.0):
    shifted_logits = logits[..., :-1, :].contiguous() / temperature
    # print(shifted_logits)
    shifted_labels = encoding.input_ids[..., 1:].contiguous()
    # print(shifted_labels)
    shifted_attention_mask = encoding.attention_mask[..., 1:].contiguous()
    # print(shifted_attention_mask)
    if median:
        ce_nan = (ce_loss_fn(shifted_logits.transpose(1, 2), shifted_labels).
                  masked_fill(~shifted_attention_mask.bool(), float("nan")))
        # print(ce_nan)
        walk = ce_nan.to("cpu").float().numpy()
        # print(walk)
        ppl = np.nanmedian(ce_nan.cpu().float().numpy(), 1)

    else:
        ppl = (ce_loss_fn(shifted_logits.transpose(1, 2), shifted_labels) *
               shifted_attention_mask).sum(1) / shifted_attention_mask.sum(1)
        ppl = ppl.to("cpu").float().numpy()

    return ppl, walk
pf_ppl, pf_walk = perplexity(encodings, performer_logits, median=True)

In [27]:
def entropy(p_logits: torch.Tensor,
            q_logits: torch.Tensor,
            encoding: transformers.BatchEncoding,
            pad_token_id: int,
            median: bool = False,
            sample_p: bool = False,
            temperature: float = 1.0):
    vocab_size = p_logits.shape[-1]
    total_tokens_available = q_logits.shape[-2]
    p_scores, q_scores = p_logits / temperature, q_logits / temperature

    p_proba = softmax_fn(p_scores).view(-1, vocab_size)

    if sample_p:
        p_proba = torch.multinomial(p_proba.view(-1, vocab_size), replacement=True, num_samples=1).view(-1)

    q_scores = q_scores.view(-1, vocab_size)

    ce = ce_loss_fn(input=q_scores, target=p_proba).view(-1, total_tokens_available)
    padding_mask = (encoding.input_ids != pad_token_id).type(torch.uint8)

    if median:
        ce_nan = ce.masked_fill(~padding_mask.bool(), float("nan"))
        en_walk = ce_nan.to("cpu").float().numpy()
        agg_ce = np.nanmedian(ce_nan.cpu().float().numpy(), 1)
        return agg_ce, en_walk
    else:
        agg_ce = (((ce * padding_mask).sum(1) / padding_mask.sum(1)).to("cpu").float().numpy())

    return agg_ce
x_ppl, en_walk = entropy(observer_logits, performer_logits, encodings, pad_token, median=True)

In [31]:
bino_walk = (pf_walk / en_walk[..., :511])

In [32]:
pkl.dump(bino_walk, open('bino_walk_list.pkl', 'wb'))
files.download('bino_walk_list.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>